In [519]:
import authentication.credential as CR

ModuleNotFoundError: No module named 'authentication'

## Notion API

In [1]:
import requests
import pandas as pd

In [56]:
# import authentication.credential as cr
NOTION_TOKEN = CR.NOTION_TOKEN

In [57]:
base_url = "https://api.notion.com/v1/"
base_headers = {
    "accept": "application/json",
    "Notion-Version": "2022-06-28",
    "Authorization": NOTION_TOKEN
}

In [58]:
# search(payload) searches all original pages, databases, and child pages/databases that are shared with the integration.
def search(payload: dict={}):
    url = base_url + "search"
    headers = base_headers
    headers.update({"content-type": "application/json"})

    response = requests.post(url=url, json=payload, headers=headers)
    return response.json()

In [59]:
# database(database_id) retrieves a Database object using the ID specified.
def database(database_id: str):
    url = base_url + "databases/" + database_id

    response = requests.get(url=url, headers=base_headers)
    return response.json()

In [60]:
# query_database(database_id, payload) gets a list of Pages contained in the database, 
# fitered and ordered according to the filter conditions and sort criteria provided in the request.
def query_database(database_id: str, payload: dict = {}):
    url = base_url + "databases/" + database_id + "/query"
    headers = base_headers
    headers.update({"content-type": "application/json"})

    response = requests.post(url=url, json=payload, headers=headers)
    return response.json()

In [61]:
# page(page_id) retrieves a Page object using the ID specified.
def page(page_id: str):
    url = base_url + "pages/" + page_id

    response = requests.get(url=url, headers=base_headers)
    return response.json()

In [62]:
# block(block_id) retrieves a Block object using the ID specified.
def block(block_id: str):
    url = base_url + "blocks/" + block_id

    response = requests.get(url=url, headers=base_headers)
    return response.json()

In [63]:
# block_children(block_id) returns a paginated array of children block objects contained in the block using
# the ID specified.
def block_children(block_id: str):
    url = base_url + "blocks/" + block_id + "/children"

    response = requests.get(url=url, headers=base_headers)
    return response.json()


In [75]:
# get_timetables() returns a list of Block objects for Timetable databases
def get_timetables():

    # Page ID for 'Year 2023' that contains the WEEKS pages.
    page_id = "513f9a9b1ebc44f28a632db0b88c2ac7"

    # filter the 'Year 2023' page for WEEKS pages
    filter_for_weeks = lambda x: (x['type'] == 'child_page') and ('WEEK' in x['child_page']['title'])
    children = filter(filter_for_weeks, block_children(page_id)['results'])

    # filter the WEEKS pages for Timetable databases
    filter_for_timetables = lambda x: (x['type'] == 'child_database') and (x['child_database']['title'] == 'Timetable')
    result = [(child['child_page']['title'], next(filter(filter_for_timetables, block_children(child['id'])['results']))['id']) for child in children]

    
    return result

In [66]:
def extract_action(data: list):
    return "" if data == [] else data[0]["text"]["content"]

In [67]:
def notion_to_pandas(database_id: str):

    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    week = query_database(database_id, payload={"sorts":[{"property": "Time", "direction": "ascending"}]})['results']
    data = list(map(lambda x: [x["properties"]["Time"]["title"][0]["text"]["content"]] + [extract_action(x["properties"][day]["rich_text"]) for day in days], week))
    return pd.DataFrame(data, columns=(['Time'] + days))

## PocketBase API

In [366]:
import datetime
import requests
import pandas as pd
# from pocketbase import PocketBase (This library is currently out-dated)

In [105]:
base_url = "http://127.0.0.1:8090"
ADMIN_EMAIL = CR.POCKETBASE_ADMIN_EMAIL
ADMIN_PASSWORD = CR.POCKETBASE_ADMIN_PASSWORD
POCKETBASE_AUTH_TOKEN = CR.POCKETBASE_AUTH_TOKEN

In [381]:
def admin_auth_with_password():

    url = base_url + "/api/admins/auth-with-password"
    body = {"identity": ADMIN_EMAIL, "password": ADMIN_PASSWORD}
    response = requests.post(url=url, json=body, params={}, headers={})
    return response

In [382]:
def get_levels(level: str, params: dict = {}, full: bool = True):

    url = base_url + f"/api/collections/{level}/records"
    headers = {"Authorization": POCKETBASE_AUTH_TOKEN}
    response = requests.get(url=url, params=params, headers=headers)
    if (full):
        return response
    else:
        items = response.json()['items']
        reduced_items = map(lambda x: {x['classification']: x['id']}, items)
        result = {}
        for x in list(reduced_items):
            result.update(x)
        return result

In [383]:
def post_level(level: str, json: dict, full: bool = True):

    url = base_url + f"/api/collections/{level}/records"
    headers = {"Authorization": POCKETBASE_AUTH_TOKEN}
    response = requests.post(url=url, json=json, params={}, headers=headers)
    if (full):
        return response
    else:
        return {response.json()['classification']: response.json()['id']}

In [384]:
def get_actions(params: dict = {}, full: bool = True):

    url = base_url + "/api/collections/actions/records"
    headers = {"Authorization": POCKETBASE_AUTH_TOKEN}
    response = requests.get(url=url, params=params, headers=headers)
    if (full):
        return response
    else:
        items = response.json()['items']
        reduced_items = map(lambda x: {x['action']: x['id']}, items)
        result = {}
        for x in list(reduced_items):
            result.update(x)
        return result

In [385]:
def post_action(json: dict):

    url = base_url + "/api/collections/actions/records"
    headers = {"Authorization": POCKETBASE_AUTH_TOKEN}
    response = requests.post(url=url, json=json, params={}, headers=headers)
    return response

In [386]:
def patch_action(id: str, json: dict):

    url = base_url + f"/api/collections/actions/records/{id}"
    headers = {"Authorization": POCKETBASE_AUTH_TOKEN}
    response = requests.patch(url=url, json=json, headers=headers)
    return response

In [478]:
def get_action_by_name(name: str):

    # If the action already exists in the database
    if ((pre_action := get_actions(params={"filter": f'action="{name}"'}, full=False)) != {}):
        return pre_action
    # If the action does not exist in the database
    else:
        data = post_action(json={"action": name}).json()
        return {data['action']: data['id']}

In [443]:
def get_eras(params: dict, full: bool = True):

    url = base_url + "/api/collections/eras/records"
    headers = {"Authorization": POCKETBASE_AUTH_TOKEN}
    response = requests.get(url=url, params=params, headers=headers)
    if (full):
        return response
    else:
        items = response.json()['items']
        reduced_items = map(lambda x: {x['era']: x['id']}, items)
        result = {}
        for x in list(reduced_items):
            result.update(x)
        return result

In [388]:
def post_era(json: dict):

    url = base_url + "/api/collections/eras/records"
    headers = {"Authorization": POCKETBASE_AUTH_TOKEN}
    response = requests.post(url=url, json=json, params={}, headers=headers)
    return response

In [389]:
def get_days():
    pass

In [390]:
def post_day(json: dict):

    url = base_url + '/api/collections/days/records'
    headers = {"Authorization": POCKETBASE_AUTH_TOKEN}
    response = requests.post(url=url, json=json, headers=headers)
    return response

In [391]:
import json
def read_json(filePath):
    file = open(filePath)
    return json.load(file)

In [512]:
fall2021 = read_json("../database/weeks_data/json_data/fall2021.json")
spring2022 = read_json("../database/weeks_data/json_data/spring2022.json")

In [393]:
# populate_database(level, data) will update the database with the actions found
# in data. `level` is used to gather the classifications. `data` should be in the 
# form of a dictionary with keys as classifications for the given `level` and values
# as lists of actions.
def populate_database(level: str, data: dict):

    # Get the current classifications for the level
    cats = get_levels(level=level, full=False)

    # Loop through all the classifications
    for cat in list(data.keys()):

        # Get the set of all actions
        # actions = set(data[cat]["weekday"] + data[cat]["weekend"])
        actions = set(data[cat])

        # Loop through all the actions
        for action in actions:

            # If the classication in NOT in the level's collection
            if (cats.get(cat, 0) == 0):
                # Add the classification to the level's collection
                new_cat = post_level(level=level, json={"classification": cat}, full=False)
                # Update the current classifications
                cats.update(new_cat)

            # If the action is already in the collection
            if ((pre_action := get_actions(params={"filter": f'action="{action}"'}, full=False)) != {}):
                # Patch the action with the current level's classification
                patch_action(pre_action[action], json={level: cats[cat]})
                continue

            # Update the action's collection
            post_action({"action": action, level: cats[cat]})

    

In [215]:
# Pick the level
level = 'level_three'
# Get the level's data
data = spring2022['classification'][level]['Work']['CS 4411']

# populate_database(level=level, data=data)

## DataFrame to Database

In [517]:
import src.recap as RP
import src.extra as EX
import datetime
from IPython.display import clear_output

In [447]:
def create_eastern_datetime(dt_date: datetime.datetime, time: str):

    # Eastern Standard Time
    timezone = datetime.timezone(offset=datetime.timedelta(hours=-5), name='EST')

    # Check if EST is correct for date range, if false, switch to Eastern Daylight Time (EDT)
    check = datetime.datetime.combine(dt_date, datetime.time(tzinfo=timezone)).astimezone()
    if (check.tzinfo != timezone):
        timezone = datetime.timezone(offset=datetime.timedelta(hours=-4), name='EDT')

    # Create datetime time
    hours = int(time.split(':')[0])
    minutes = int(time.split(':')[1])
    dt_time = datetime.time(hour=hours, minute=minutes, tzinfo=timezone)

    # Create final datetime
    final_dt = datetime.datetime.combine(dt_date, dt_time)

    return final_dt

In [448]:
def text_to_date(date: str):

    # Create datetime date
    return datetime.datetime.strptime(date, "%m/%d/%Y")

In [488]:
# to_UTC(dt, text) returns a datetime object if `text` is False, or a string format in `text` is True
def to_UTC(dt: datetime.datetime, text: bool = False):

    if (text):
        return dt.astimezone(tz=datetime.timezone(offset=datetime.timedelta(hours=0))).isoformat(' ')[:-6]
    else:
        return dt.astimezone(tz=datetime.timezone(offset=datetime.timedelta(hours=0)))

In [514]:
def create_dataframes(folder: str, class_links: dict):
    
    # Defining an empty list to hold strings that represent file locations
    weeks_files = []

    # For Loop to create strings that represent the file location of exported .csv files from Notion
    for x in range(1, 16):
        try:
            file = f"../database/weeks_data/{folder}/Week{str(x)}.csv"
            weeks_files.append(file)
        except:
            continue

    # Defining an empty list to hold Pandas Dataframe of the previously mentioned Notion .csv files
    weeks_df = []

    # For Loop to create, and clean multipe Dataframes to represent a week
    for x in range(15):
        try:
            df = pd.read_csv(weeks_files[x])
            time = pd.DataFrame(EX.military_time())
            df = df.drop(df.columns[[0]], axis=1)
            df = pd.concat([time, df], axis=1)
            df.columns = pd.Index(['Time', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday'], dtype='object')
            df = df.replace(class_links)
            df = df.replace(to_replace=r'^https://www.notion.so/.*$', value="PIKE Meeting", regex=True)
            weeks_df.append(df.fillna("Unknown"))
        except Exception:
            continue

    return weeks_df

In [515]:
class_links = spring2022['class_link']
begin = spring2022['metadata']['date_start']
end = spring2022['metadata']['date_end']
folder = "spring2022"
era = "Spring 2022"

dataframes = create_dataframes(folder=folder, class_links=class_links)

days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']

split_index = 72

eras = get_eras(params={}, full=False)

In [518]:
# Define the starting datetime date
dt_date = text_to_date(date=begin)

# Define pointer variables
front_actions = pd.DataFrame() # actions from 00:00 to 05:45
back_actions = [] # actions from 06:00 to 23:45

# Loop through all weeks
for week in dataframes:

    past_day = 'Sunday'

    # Loop through all days
    for day_index, day in enumerate(days):

        # Check if there are actions from 00:00 to 05:45
        if (front_actions.shape[0] != 0):
            
            # Loop through all actions from 00:00 to 05:45
            for index, time_action in front_actions.iterrows():

                # Create time-string
                eastern = create_eastern_datetime(dt_date=dt_date, time=time_action['Time'])
                utc = to_UTC(dt=eastern, text=True)
                # Get action's ID
                action_name = time_action[past_day]
                action_dict = get_action_by_name(name=action_name)
                # Post day with date-string, era_ID, and action_ID
                clear_output(wait=True)
                print(f"Processing date {utc}")
                post_day(json={"date": utc, "era": eras[era], "action": action_dict[action_name]})


        back_actions = week[['Time', day]].iloc[:split_index]

        # Loop through all actions from 06:00 to 23:45
        for index, time_action in back_actions.iterrows():

            # Create time-string
            eastern = create_eastern_datetime(dt_date=dt_date, time=time_action['Time'])
            utc = to_UTC(dt=eastern, text=True)
            # Get action's ID
            action_name = (time_action[day]).replace('"', "'")
            action_dict = get_action_by_name(name=action_name)
            # Post day with date-string, era_ID, and action_ID
            clear_output(wait=True)
            print(f"Processing date {utc}")
            post_day(json={"date": utc, "era": eras[era], "action": action_dict[action_name]})

        # Update front_actions
        front_actions = week[['Time', day]].iloc[split_index:]
        # Update datetime date
        dt_date = dt_date + datetime.timedelta(days=1)
        # Update past day
        past_day = day

# Final Loop through front_actions pointer
# Loop through all actions from 00:00 to 05:45
for index, time_action in front_actions.iterrows():

    # Create time-string
    eastern = create_eastern_datetime(dt_date=dt_date, time=time_action['Time'])
    utc = to_UTC(dt=eastern, text=True)
    # Get action's ID
    action_name = time_action[past_day]
    action_dict = get_action_by_name(name=action_name)
    # Post day with date-string, era_ID, and action_ID
    post_day(json={"date": utc, "era": eras[era], "action": action_dict[action_name]})

Processing date 2022-05-09 03:45:00
